# Subtask 1
# X-Means and K-Medoids Clustering


Requirements:

- Python 3.13.1
- numpy
- pandas
- scikit-learn
- matplotlib
- pyclustering


In [ ]:
import logging
from typing import List, Optional, Dict

import numpy as np
import pandas as pd
from pandas.api.types import is_integer_dtype
from sklearn.calibration import LabelEncoder
from sklearn.discriminant_analysis import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# Pyclustering imports
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.kmedoids import kmedoids
from pyclustering.utils.metric import distance_metric, type_metric


## Preprocessing
Load the data and preprocess it. Drop features we dont need. Since we are working with mixed-type data, we need to encode features that are not continuous numeric. Also, missing values need to be imputated.


In [ ]:
df = pd.read_csv("../dataset/tracks.csv")
df.head()


### Drop Irrelevant Features
Many columns such as names or unique identifiers are not relevant for clustering. We drop them.


In [ ]:
drop_cols = [
    "id",
    "id_artist",
    "name_artist",
    "full_title",
    "title",
    "swear_IT_words",
    "swear_EN_words",
    "album_name",
    "album_release_date",
    "id_album",
    "album_image",
    "lyrics",
    "month",
    "day",
    "disc_number",
    "track_number",
    "album_type",
    "popularity",
    "year",
    "primary_artist",
    "featured_artists",
    "album",
]

df = df.drop(columns=drop_cols)
df.head()


In [ ]:
# Relevant preprocessing functions

def clean(df: pd.DataFrame) -> pd.DataFrame:
    # Replace empty cells (or ?) with NA
    df = df.replace(r"^\?|\s+$", pd.NA, regex=True)

    # Columns with whitespace (or ?) values are infered as string type but could be numeric
    for col in df.select_dtypes(exclude=[np.number, np.bool_]).columns:
        # Exclude booleans
        if df[col].nunique() > 2:
            try:
                df[col] = pd.to_numeric(df[col])
            except ValueError:
                logging.debug("Could not convert column %s to numeric dtype", col)

    # Integer columns with missing values automatically get converted to float columns
    # We want to convert back to integer to allow for better column classification (num/cat)
    for col in df.select_dtypes(include=np.floating):
        if np.all(df[col].fillna(0) % 1 == 0):
            df[col] = df[col].astype("Int64")

    # Turn object columns into string columns
    for col in df.select_dtypes(include=object).columns:
        df[col] = df[col].astype(str)

    return df


def scale_cols(
    df: pd.DataFrame,
    num_cols: Optional[List[str]],
    cat_cols: Optional[List[str]],
    bool_cols: Optional[List[str]] = None,
) -> pd.DataFrame:
    df = df.copy(deep=True)
    if num_cols:
        df[num_cols] = StandardScaler().fit_transform(df[num_cols])
    if cat_cols:
        df[cat_cols] = df[cat_cols].apply(LabelEncoder().fit_transform)
    if bool_cols:
        df[bool_cols] = df[bool_cols].apply(LabelEncoder().fit_transform)
    return df


def imputate_na(
    df: pd.DataFrame,
    num_cols: Optional[List[str]],
    cat_cols: Optional[List[str]],
    bool_cols: Optional[List[str]] = None,
) -> pd.DataFrame:
    if num_cols:
        # imputate numeric features
        for col in num_cols:
            # For integer columns we need to round the median
            if is_integer_dtype(df[col]):
                df[col] = df[col].fillna(round(df[col].median()))
            else:
                df[col] = df[col].fillna(df[col].median())
    if cat_cols is None:
        cat_cols = []
    if bool_cols is None:
        bool_cols = []
    # add "missing" category for categorical features
    for col in cat_cols + bool_cols:
        df[col] = df[col].astype(str)
        df[col] = df[col].fillna("<NA>")
    return df


### Cleaning, Imputating, Scaling and Labelling
Now execute the functions from above to return a dataframe we can use for computation.


In [ ]:
# Save old dataframe for later
og_df = df.copy(deep=True)

cat_cols = [
    "language",
]
bool_cols = ["explicit"]
num_cols = [
    "stats_pageviews",
    "n_sentences",
    "n_tokens",
    "swear_IT",
    "swear_EN",
    "tokens_per_sent",
    "char_per_tok",
    "lexical_density",
    "avg_token_per_clause",
    "bpm",
    "centroid",
    "rolloff",
    "flux",
    "rms",
    "zcr",
    "flatness",
    "spectral_complexity",
    "pitch",
    "loudness",
    "duration_ms",
    "modified_popularity",
]

# Check we registered all columns
assert set(cat_cols + bool_cols + num_cols) == set(df.columns)

df = clean(df)
df = imputate_na(df, num_cols, cat_cols, bool_cols)
df = scale_cols(df, num_cols, cat_cols, bool_cols)

df.head()


### Encoding
Now we use one-hot encoding to encode the categorical features. Boolean features only need encoding if they contain NaN or NA values (=missing values), otherwise they are left as is. To reduce the number of features, we only encode features that have more unique values than a threshhold. This will get rid of features with too many unique values, such as song lyrics. These features would add up to N new features to the dataset, which is not a good idea.


In [ ]:
def one_hot_encode_feature(df: pd.DataFrame, feature_to_encode: str, weight: float = 1) -> pd.DataFrame:
    dummies = pd.get_dummies(df[feature_to_encode], dtype="int32", prefix=feature_to_encode)
    dummies *= weight
    result_df = pd.concat([df, dummies], axis=1)
    return result_df.drop(columns=feature_to_encode)


encoded_df = df.copy(deep=True)

for col in cat_cols + [col for col in bool_cols if encoded_df[col].nunique() > 2]:
    # Dont encode columns with too many unique values
    if encoded_df[col].nunique() > 50:
        logging.warning("Column %s has too many unique values, dropping", col)
        encoded_df = encoded_df.drop(columns=col)
    else:
        encoded_df = one_hot_encode_feature(encoded_df, col)

encoded_df.head()


## X-Means Clustering
X-means is an extension of k-means that automatically determines the optimal number of clusters. It starts with k=2 and uses statistical tests to decide whether to split clusters or stop. This eliminates the need to manually specify the number of clusters.

### X-Means Algorithm
X-means clustering is performed and evaluated using the same metrics as k-means:
- **Sum of Squared Errors (SSE)**: Lower is better
- **Silhouette Score**: Higher is better  
- **Variance Ratio Criterion**: Higher is better
- **Davies-Bouldin Index**: Lower is better


In [ ]:
# Convert data to list format for pyclustering
data = encoded_df.values.tolist()

# X-means clustering
xmeans_instance = xmeans(data, k_max=10)
xmeans_instance.process()

# Get results
xmeans_centers = xmeans_instance.get_centers()
xmeans_clusters = xmeans_instance.get_clusters()
xmeans_labels = np.zeros(len(data), dtype=int)

# Convert cluster format to labels
for cluster_id, cluster in enumerate(xmeans_clusters):
    for point_idx in cluster:
        xmeans_labels[point_idx] = cluster_id

print(f"X-means found {len(xmeans_centers)} clusters")
print(f"Cluster sizes: {[len(cluster) for cluster in xmeans_clusters]}")

# Calculate metrics for X-means
xmeans_sse = 0
for cluster_id, cluster in enumerate(xmeans_clusters):
    center = xmeans_centers[cluster_id]
    for point_idx in cluster:
        point = data[point_idx]
        xmeans_sse += sum((np.array(point) - np.array(center))**2)

xmeans_silhouette = silhouette_score(encoded_df.values, xmeans_labels)
xmeans_vrc = calinski_harabasz_score(encoded_df.values, xmeans_labels)
xmeans_davies_bouldin = davies_bouldin_score(encoded_df.values, xmeans_labels)

print(f"X-means SSE: {xmeans_sse:.2f}")
print(f"X-means Silhouette: {xmeans_silhouette:.3f}")
print(f"X-means VRC: {xmeans_vrc:.2f}")
print(f"X-means Davies-Bouldin: {xmeans_davies_bouldin:.3f}")


## K-Medoids Clustering
K-medoids is similar to k-means but uses actual data points (medoids) as cluster centers instead of centroids. This makes it more robust to outliers. We'll test different values of k and compare the results.


In [ ]:
# K-medoids clustering for different k values
kmedoids_metrics = {
    'sse': [],
    'silhouette': [],
    'vrc': [],
    'davies_bouldin': []
}
kmedoids_results = {}

for k in range(2, 11):
    # Initialize k-medoids with random medoids
    initial_medoids = np.random.choice(len(data), k, replace=False).tolist()
    
    # Create k-medoids instance
    kmedoids_instance = kmedoids(data, initial_medoids)
    kmedoids_instance.process()
    
    # Get results
    kmedoids_medoids = kmedoids_instance.get_medoids()
    kmedoids_clusters = kmedoids_instance.get_clusters()
    kmedoids_labels = np.zeros(len(data), dtype=int)
    
    # Convert cluster format to labels
    for cluster_id, cluster in enumerate(kmedoids_clusters):
        for point_idx in cluster:
            kmedoids_labels[point_idx] = cluster_id
    
    kmedoids_results[k] = kmedoids_labels
    
    # Calculate SSE for k-medoids
    kmedoids_sse = 0
    for cluster_id, cluster in enumerate(kmedoids_clusters):
        medoid_idx = kmedoids_medoids[cluster_id]
        medoid = data[medoid_idx]
        for point_idx in cluster:
            point = data[point_idx]
            kmedoids_sse += sum((np.array(point) - np.array(medoid))**2)
    
    kmedoids_metrics['sse'].append(kmedoids_sse)
    kmedoids_metrics['silhouette'].append(silhouette_score(encoded_df.values, kmedoids_labels))
    kmedoids_metrics['vrc'].append(calinski_harabasz_score(encoded_df.values, kmedoids_labels))
    kmedoids_metrics['davies_bouldin'].append(davies_bouldin_score(encoded_df.values, kmedoids_labels))

# Plot k-medoids metrics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
k_range = range(2, 11)

axes[0,0].plot(k_range, kmedoids_metrics['sse'], 'bo-')
axes[0,0].set_title('K-Medoids SSE (Lower is better)')
axes[0,0].set_xlabel('k')

axes[0,1].plot(k_range, kmedoids_metrics['silhouette'], 'ro-')
axes[0,1].set_title('K-Medoids Silhouette Score (Higher is better)')
axes[0,1].set_xlabel('k')

axes[1,0].plot(k_range, kmedoids_metrics['vrc'], 'go-')
axes[1,0].set_title('K-Medoids Variance Ratio Criterion (Higher is better)')
axes[1,0].set_xlabel('k')

axes[1,1].plot(k_range, kmedoids_metrics['davies_bouldin'], 'mo-')
axes[1,1].set_title('K-Medoids Davies-Bouldin Index (Lower is better)')
axes[1,1].set_xlabel('k')

plt.tight_layout()
plt.show()


## Algorithm Comparison
Let's compare the performance of X-means and K-medoids algorithms by plotting their metrics side by side.


In [ ]:
# Compare X-means with K-medoids
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Find best k for k-medoids based on silhouette score
best_k = k_range[np.argmax(kmedoids_metrics['silhouette'])]
print(f"Best k for K-medoids based on silhouette score: {best_k}")

# Plot comparison
axes[0,0].plot([len(xmeans_centers)], [xmeans_sse], 'ro', markersize=10, label='X-means')
axes[0,0].plot(k_range, kmedoids_metrics['sse'], 'bo-', label='K-medoids')
axes[0,0].set_title('SSE Comparison')
axes[0,0].set_xlabel('k')
axes[0,0].legend()

axes[0,1].plot([len(xmeans_centers)], [xmeans_silhouette], 'ro', markersize=10, label='X-means')
axes[0,1].plot(k_range, kmedoids_metrics['silhouette'], 'bo-', label='K-medoids')
axes[0,1].set_title('Silhouette Score Comparison')
axes[0,1].set_xlabel('k')
axes[0,1].legend()

axes[1,0].plot([len(xmeans_centers)], [xmeans_vrc], 'ro', markersize=10, label='X-means')
axes[1,0].plot(k_range, kmedoids_metrics['vrc'], 'bo-', label='K-medoids')
axes[1,0].set_title('Variance Ratio Criterion Comparison')
axes[1,0].set_xlabel('k')
axes[1,0].legend()

axes[1,1].plot([len(xmeans_centers)], [xmeans_davies_bouldin], 'ro', markersize=10, label='X-means')
axes[1,1].plot(k_range, kmedoids_metrics['davies_bouldin'], 'bo-', label='K-medoids')
axes[1,1].set_title('Davies-Bouldin Index Comparison')
axes[1,1].set_xlabel('k')
axes[1,1].legend()

plt.tight_layout()
plt.show()

print(f"\nX-means Results:")
print(f"  Number of clusters: {len(xmeans_centers)}")
print(f"  SSE: {xmeans_sse:.2f}")
print(f"  Silhouette: {xmeans_silhouette:.3f}")
print(f"  VRC: {xmeans_vrc:.2f}")
print(f"  Davies-Bouldin: {xmeans_davies_bouldin:.3f}")

print(f"\nK-medoids Results (k={best_k}):")
print(f"  SSE: {kmedoids_metrics['sse'][best_k-2]:.2f}")
print(f"  Silhouette: {kmedoids_metrics['silhouette'][best_k-2]:.3f}")
print(f"  VRC: {kmedoids_metrics['vrc'][best_k-2]:.2f}")
print(f"  Davies-Bouldin: {kmedoids_metrics['davies_bouldin'][best_k-2]:.3f}")


## Evaluation
### Cluster Overview
What do we do with the clusters now? What we can do is to look into each cluster and look into the average variables that it has. We create a table that has a row for each cluster and a column for each variable. The value in the cell is the rounded average of that variable for that cluster. In case of categorical variables, the most common category is listed with its percentage in braces. For boolean variables, both options are listed with their percentages. The columns (features) are sorted by their importance towards the clustering process, more on that in the next section.


In [ ]:
def cluster_overview(
    original_df: pd.DataFrame,
    cluster_labels: np.ndarray,
    num_cols: List[str],
    cat_cols: List[str],
    bool_cols: List[str],
    feature_importances: Dict[str, float],
) -> pd.DataFrame:
    def num_agg(series):
        res = f"~{series.mean():.2f}"
        if res == "~-0.0":
            return "~0.0"
        return res

    def cat_agg(series):
        vc = series.value_counts(normalize=True)
        if len(vc) >= 1:
            return f"{vc.index[0]} ({vc.iloc[0]:.0%})"
        else:
            return "NA (100%)"

    def bool_agg(series):
        vc = series.value_counts(normalize=True)
        if len(vc) == 1:
            return f"{vc.iloc[0]:.0%} {vc.index[0]}"
        elif len(vc) == 2:
            return f"{vc.iloc[0]:.0%} {vc.index[0]}/{vc.iloc[1]:.0%} {vc.index[1]}"
        else:
            return "100% NA"

    df = original_df.copy(deep=True)
    df["Cluster ID"] = cluster_labels
    
    # Calculate per-cluster aggregations
    df_clusters = pd.concat(
        [
            df[num_cols + ["Cluster ID"]].groupby("Cluster ID").agg(num_agg) if num_cols else pd.DataFrame(),
            df[cat_cols + ["Cluster ID"]].groupby("Cluster ID").agg(cat_agg) if cat_cols else pd.DataFrame(),
            df[bool_cols + ["Cluster ID"]].groupby("Cluster ID").agg(bool_agg) if bool_cols else pd.DataFrame(),
        ],
        axis=1,
    )
    df_clusters = df_clusters.reset_index()
    
    # Calculate overall aggregations for entire dataset
    df_overall = pd.concat(
        [
            df[num_cols].agg(num_agg).to_frame().T if num_cols else pd.DataFrame(),
            df[cat_cols].agg(cat_agg).to_frame().T if cat_cols else pd.DataFrame(),
            df[bool_cols].agg(bool_agg).to_frame().T if bool_cols else pd.DataFrame(),
        ],
        axis=1,
    )
    df_overall["Cluster ID"] = "All Clusters"
    
    # Add member counts
    counts = np.bincount(cluster_labels)
    df_clusters["Members"] = df_clusters.apply(lambda row: counts[int(row["Cluster ID"])], axis=1)
    df_overall["Members"] = len(cluster_labels)
    
    # Format cluster IDs
    df_clusters["Cluster ID"] = df_clusters["Cluster ID"].apply(lambda x: f"Cluster {x}")
    
    # Combine cluster data with overall data
    df_result = pd.concat([df_clusters, df_overall], ignore_index=True)
    
    # Order features by importance
    df_result = df_result[["Cluster ID", "Members"] + list(feature_importances.keys())]

    return df_result


def feature_importances(
    df: pd.DataFrame,
    labels: np.ndarray
) -> Dict[str, float]:
    classifier = RandomForestClassifier(n_estimators=100)
    classifier.fit(df, labels)
    return dict(
        sorted(
            zip(df.columns, classifier.feature_importances_),
            key=lambda it: it[1],
            reverse=True,
        )
    )


### X-Means Cluster Analysis


In [ ]:
# X-means cluster analysis
xmeans_feature_importances = feature_importances(encoded_df, xmeans_labels)
xmeans_cluster_overview = cluster_overview(og_df, xmeans_labels, num_cols, cat_cols, bool_cols, xmeans_feature_importances)
print("X-Means Cluster Overview:")
xmeans_cluster_overview


### K-Medoids Cluster Analysis


In [ ]:
# K-medoids cluster analysis (using best k)
kmedoids_labels = kmedoids_results[best_k]
kmedoids_feature_importances = feature_importances(encoded_df, kmedoids_labels)
kmedoids_cluster_overview = cluster_overview(og_df, kmedoids_labels, num_cols, cat_cols, bool_cols, kmedoids_feature_importances)
print(f"K-Medoids Cluster Overview (k={best_k}):")
kmedoids_cluster_overview


### Feature Importances
The feature importance was already used above to sort the columns of the cluster overview table descending by their importance towards the clustering process. We can also plot it as a bar chart to get a better understanding of the features that contributed most to the separation of the clusters. 

We cannot directly extract the feature importance from the clustering models, but instead we can approximate the feature importances by training a supervised model on the cluster labels we generated. Some sort of decision tree models works well because of their explainability, which allows us to calculate the feature importances. We use a Random Forest Classifier here.


In [ ]:
# Feature importance comparison
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# X-means feature importance
top_n = 20
xmeans_top_features = list(xmeans_feature_importances.items())[:top_n]
xmeans_features, xmeans_importances = zip(*xmeans_top_features)

axes[0].barh(range(len(xmeans_features)), xmeans_importances, color='steelblue')
axes[0].set_yticks(range(len(xmeans_features)))
axes[0].set_yticklabels(xmeans_features)
axes[0].set_xlabel('Importance')
axes[0].set_ylabel('Feature')
axes[0].set_title(f'Top {top_n} Feature Importances for X-Means Clustering')
axes[0].invert_yaxis()

# K-medoids feature importance
kmedoids_top_features = list(kmedoids_feature_importances.items())[:top_n]
kmedoids_features, kmedoids_importances = zip(*kmedoids_top_features)

axes[1].barh(range(len(kmedoids_features)), kmedoids_importances, color='darkorange')
axes[1].set_yticks(range(len(kmedoids_features)))
axes[1].set_yticklabels(kmedoids_features)
axes[1].set_xlabel('Importance')
axes[1].set_ylabel('Feature')
axes[1].set_title(f'Top {top_n} Feature Importances for K-Medoids Clustering (k={best_k})')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()


### Artists per Cluster
To validate whether the clustering makes sense, we can examine which artists belong to each cluster. This allows us to listen to the artists and verify if the clusters have meaningful musical or stylistic coherence.


In [ ]:
def get_artists_per_cluster(
    original_df: pd.DataFrame,
    cluster_labels: np.ndarray,
    artist_column: str = 'name_artist',
    top_n: int = 10
) -> pd.DataFrame:
    original_df['cluster'] = cluster_labels
    
    cluster_data = {}
    for cluster_id in sorted(np.unique(cluster_labels)):
        cluster_df = original_df[original_df['cluster'] == cluster_id]
        
        # Count tracks per artist in this cluster
        artist_counts = cluster_df[artist_column].value_counts().head(top_n)
        
        # Format as "Artist (N tracks)"
        formatted_artists = [f"{artist} ({count})" for artist, count in zip(artist_counts.index, artist_counts.values)]
        
        # Pad with empty strings if less than top_n artists
        while len(formatted_artists) < top_n:
            formatted_artists.append("")
        
        cluster_data[f'Cluster {cluster_id}'] = formatted_artists
    
    df = pd.DataFrame(cluster_data)
    return df


# Load full dataset for artist analysis
full_og_df = pd.read_csv('../dataset/tracks.csv')

# X-means artists analysis
print("X-Means Artists per Cluster:")
xmeans_artists_df = get_artists_per_cluster(full_og_df, xmeans_labels, top_n=10)
xmeans_artists_df


In [ ]:
# K-medoids artists analysis
print(f"K-Medoids Artists per Cluster (k={best_k}):")
kmedoids_artists_df = get_artists_per_cluster(full_og_df, kmedoids_labels, top_n=10)
kmedoids_artists_df
